# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from env.api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
# Store filepath in a variable
input_file = "../WeatherPy/output_data/cities.csv"
cities_df = pd.read_csv(input_file)
cities_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,san patricio,86.00,MX,1579738050,74,19.22,-104.70,86.00,11.41
1,barrow,-2.20,US,1579738226,77,71.29,-156.79,-2.20,10.29
2,hasaki,41.00,JP,1579738213,86,35.73,140.83,41.00,1.12
3,punta arenas,51.80,CL,1579738060,71,-53.15,-70.92,51.80,12.75
4,hermanus,70.00,ZA,1579738003,85,-34.42,19.23,70.00,8.99
...,...,...,...,...,...,...,...,...,...
561,lata,48.20,PT,1579738949,93,40.16,-8.33,48.20,4.70
562,vadso,23.00,NO,1579738949,73,70.07,29.75,23.00,33.33
563,prince rupert,44.60,CA,1579738847,75,54.32,-130.32,44.60,10.29
564,acarau,76.95,BR,1579738949,90,-2.89,-40.12,76.95,3.62


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
# Access maps with API key
gmaps.configure(api_key=g_key)
locations = cities_df[["Lat", "Lng"]]
rating = cities_df["Humidity"].astype(float)

In [14]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig



Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [54]:
df = cities_df.copy()
df = df.loc[(df['Max Temp'] < 90) & (df['Max Temp'] > 58) & (df['Wind Speed'] < 20) & (df['Cloudiness'] < 60)]
df = df.dropna(how='any')
df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
42,ziway,59.56,ET,1579738229,71,7.93,38.72,59.56,7.76
69,castro,58.77,BR,1579738292,92,-24.79,-50.01,58.77,8.50
98,kaitangata,58.08,NZ,1579738017,79,-46.28,169.85,58.08,16.22
137,ayabaca,59.63,PE,1579738359,96,-4.64,-79.71,59.63,1.72
216,carbonia,59.00,IT,1579738488,87,39.17,8.52,59.00,19.46
265,eureka,59.00,US,1579738470,67,40.80,-124.16,59.00,6.93
279,mosquera,59.00,CO,1579738554,72,4.71,-74.23,59.00,6.93
319,fortuna,59.00,US,1579738329,71,40.60,-124.16,59.00,4.70
335,san quintin,59.14,MX,1579738625,77,30.48,-115.95,59.14,15.08
361,jobat,59.02,IN,1579738688,48,22.42,74.57,59.02,7.90


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display Map